# LGBM and Downsampling using spreted models 


 Score in privet leaderbord is : 0.75065 

Score in public leaderbord is : 0.71057 

### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import ndcg_score
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import lightgbm as lgb
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path
from scipy.stats import hmean
import scipy
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LassoCV



### preprocessing

In [ ]:
"""
just drop the path of your new data directory nothing more than that. 

"""

path = r'C:\Users\Family\Downloads\widsdatathon2025 (3)'

def read_data(base_path:str) -> pd.DataFrame :
    path = Path(base_path)
    trc=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_CATEGORICAL_METADATA_new.xlsx')
    trq=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_QUANTITATIVE_METADATA_new.xlsx')
    trf=pd.read_csv(path   /'TRAIN_NEW'  / 'TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
    trs=pd.read_excel(path /'TRAIN_NEW'  / 'TRAINING_SOLUTIONS.xlsx')  
    tsc=pd.read_excel(path /'TEST'      / 'TEST_CATEGORICAL.xlsx')
    tsq=pd.read_excel(path /'TEST'       / 'TEST_QUANTITATIVE_METADATA.xlsx')    
    tsf=pd.read_csv(path   /'TEST'       / 'TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')    
    sub=pd.read_excel(path / 'SAMPLE_SUBMISSION.xlsx')    
    dic=pd.read_excel(path /'Data Dictionary.xlsx')
    return trc, trq, trf, trs, tsc, tsq, tsf, sub, dic

trc, trq, trf, trs, tsc, tsq, tsf, sub, dic = read_data(base_path=path)

# Data Merging 
cq = pd.merge(trc, trq, on='participant_id', how='left')
feat = pd.merge(cq, trf, on='participant_id', how='left')  
qc = pd.merge(tsc, tsq, on='participant_id', how='left')
train = pd.merge(feat, trs, on='participant_id', how='left') 
test = pd.merge(qc, tsf, on='participant_id', how='left')
train_sex =train

In [ ]:
train_ids = train['participant_id']
test_ids = test['participant_id'] # I will store them for later usage in grouping in validation why?  I don't want the same user to appear in both train and test. 
num_feats = trq # numerical features
cat_feats = trc # seperate categorical and numerical features help me reteriving them later easily for preprocessing.
target_cols = ['ADHD_Outcome', 'Sex_F']
groups = train_ids



In [ ]:
'''
Train has 25 features/columns with missing values: ['PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ', 'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan']
Test  has 23 features/columns with missing values: ['PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ', 'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial']
fMRI has no missing values
Extra columns in train: ['MRI_Track_Age_at_Scan', 'MRI_Track_Scan_Location']
'''

# Find columns with missing values only
train_missing_features_to_impute = train.columns[train.isnull().any()].tolist() # List of features with missing values in train, only 25 and no missing data in fMRI data
test_missing_features_to_impute = test.columns[test.isnull().any()].tolist() # List of features with missing values in test, only 23 and no missing data in fMRI data


# Initialize the imputer
imputer = IterativeImputer(estimator=LassoCV(random_state=42), max_iter=5, random_state=42)

# Impute in-place for train
if train_missing_features_to_impute:
	train[train_missing_features_to_impute] = imputer.fit_transform(train[train_missing_features_to_impute])

# Impute in-place for test
if test_missing_features_to_impute:
	test[test_missing_features_to_impute] = imputer.fit_transform(test[test_missing_features_to_impute])

c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.220e+00, tolerance: 4.923e-01
  model = cd_fast.enet_coordinate_descent(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.402e+00, tolerance: 5.646e-01
  model = cd_fast.enet_coordinate_descent(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

In [ ]:
train.isnull().sum().sum(), test.isnull().sum().sum() # Check if there are any missing values left in train and test data

(0, 0)

In [ ]:
scaler = StandardScaler()

# Only apply scaling to numerical columns that are not part of the target or categorical features
numerical_features = [col for col in train.columns if col not in target_cols and col not in cat_feats]

# Fit scaler on the numerical features of the train set and transform train and test sets
train[numerical_features] = scaler.fit_transform(train[numerical_features])  # Fit and transform for train set
test[numerical_features] = scaler.transform(test[numerical_features]) 

In [ ]:
train.isnull().sum().sum(), test.isnull().sum().sum() # Check if there are any missing values left in train and test data

(0, 0)

In [ ]:
y_sex = train['Sex_F']  
y_adhd = train['ADHD_Outcome']  


### feature importance in sex

In [ ]:
train

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ,...,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn,ADHD_Outcome,Sex_F
0,00aIpNTbG5uh,2019,4,1.0,0.0,3.0,21.0,45.000000,17.903324,32.871543,...,-1.185011,0.141726,-0.890926,-0.677929,-0.180457,-0.554190,-0.835889,-0.024380,1,0
1,00fV0OyyoLfw,2017,1,0.0,9.0,2.0,21.0,0.000000,21.000000,45.000000,...,-1.441008,-1.721379,-0.001464,0.499990,0.305932,-0.239836,-0.922257,0.310269,1,0
2,04X1eiS79T4B,2017,1,1.0,2.0,2.0,9.0,0.000000,12.387745,17.897575,...,0.172175,-0.930766,0.810557,-0.436211,0.606624,-1.385552,0.554995,0.364357,0,1
3,05ocQutkURd6,2018,1,3.0,8.0,2.0,18.0,10.000000,18.000000,0.000000,...,-0.791115,-1.136420,-0.252985,-0.394303,0.234110,0.063241,-0.648517,-0.567490,0,1
4,06YUNBA9ZRLq,2018,1,0.0,1.0,2.0,12.0,0.000000,13.845904,22.432518,...,-0.505395,-0.061681,-0.278927,-0.204557,-0.283942,-0.328775,-1.684912,-0.969062,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,zwjJWCRzKhDz,2019,4,1.0,1.0,3.0,12.0,15.136452,15.000000,5.000000,...,0.732856,1.016003,0.292482,1.063558,-0.303480,-0.081801,-1.617241,-1.109779,0,1
1209,zwXD5v17Rx01,2018,1,0.0,0.0,3.0,21.0,40.000000,21.000000,40.000000,...,1.421749,0.952747,0.605150,-1.623857,0.634374,-2.157031,-0.405763,-0.035614,1,0
1210,zWzLCi3NTBTd,2018,3,2.0,3.0,3.0,21.0,40.000000,21.000000,35.000000,...,0.400430,1.134626,-0.124110,-1.780372,0.531519,-2.287286,-1.525919,-2.118786,1,1
1211,Zy9GTHDxUbXU,2019,4,0.0,1.0,3.0,18.0,35.000000,18.000000,45.000000,...,0.007159,-0.359999,0.990114,-0.976870,-1.389992,-0.451771,-1.821987,0.961151,1,0


In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

# Assuming 'train' and 'test' are your DataFrames
# and target_cols is defined as ['ADHD_Outcome', 'Sex_F']

# Prepare features (X) - drop targets and participant_id
X = train.drop(columns=target_cols + ['participant_id'])

# Convert categorical features to dummy variables
X = pd.get_dummies(X, drop_first=True)

# Encode target (y_sex should be your target Series)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_sex)

# Initialize and fit the LGBM model for Sex prediction
model_sex = lgb.LGBMClassifier(
    class_weight='balanced',  # Important for imbalanced data
    random_state=42
)
model_sex.fit(X, y)

# Get feature importances and create a DataFrame
importance_df_sex = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model_sex.feature_importances_
}).sort_values('Importance', ascending=False)

# Filter features with importance > 0 (or set a higher threshold)
important_features_sex = importance_df_sex[importance_df_sex['Importance'] > 0]['Feature'].tolist()

# Filter the data - ensure test has same features
train_sex = train[important_features_sex]
test_sex = test[important_features_sex]

# Verify the filtered data
print(f"\nNumber of important features for Sex: {len(important_features_sex)}")
print("\nTop 10 important features for Sex prediction:")
print(importance_df_sex.head(10))

[LightGBM] [Info] Number of positive: 416, number of negative: 797
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.603047 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5075431
[LightGBM] [Info] Number of data points in the train set: 1213, number of used features: 19927
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000

Number of important features for Sex: 2310

Top 10 important features for Sex prediction:
                    Feature  Importance
19098  158throw_191thcolumn          21
16356  114throw_199thcolumn          15
13249   83throw_192thcolumn          14
17753  133throw_171thcolumn          11
18830  152throw_184thcolumn          10
14901   99throw_124thcolumn          10
19896  191throw_197thcolumn           9
19321  164throw_189thcolumn           9
7861     44throw_69thcolumn           8
12304    76thr

In [ ]:
# Instead of > 0 threshold, consider:
threshold = importance_df_sex['Importance'].quantile(0.98)  
important_features_sex = importance_df_sex[importance_df_sex['Importance'] > threshold]['Feature'].tolist()

# Modeling 


**First, I trained a separate LightGBM model for each target, using bagging techniques to handle the class imbalance**

### SEX

This code trains a LightGBM model to predict y_sex using a 5-bag, 5-fold ensemble with class balancing. In each fold, the majority class is downsampled to match the minority, helping with imbalance. Predictions are averaged across all bags and folds, and performance is measured using the macro F1-score for a fair evaluation of both classes.

In [ ]:
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

# Initialize storage
oof_sex = np.zeros(len(important_features_sex))
models_sep_sex = {}  # Initialize models_sep_sex as an empty dictionary
bag_f1_sex = []

BAGS = 5
FOLDS = 5

# Prepare data
important_features_sex_filtered = important_features_sex.reset_index(drop=True)
y_sex = y_sex.reset_index(drop=True)

for bag in range(BAGS):
    print('#'*25)
    print(f'### Bag {bag+1}')
    print('#'*25)
    
    models_sep_sex[bag] = []
    bag_pred_sex = np.zeros(len(important_features_sex))
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=bag*FOLDS)
    fold_f1_sex = []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X=important_features_sex_filtered, y=y_sex)):
        print(f'=> Fold {fold+1}, ', end='')

        # Dynamic class balancing
        y_important_features_sex = y_sex.iloc[train_idx]
        class_counts = y_important_features_sex.value_counts()
        majority_class = class_counts.idxmax()
        minority_class = 1 - majority_class
        
        majority_samples = y_important_features_sex[y_important_features_sex == majority_class]
        minority_samples = y_important_features_sex[y_important_features_sex == minority_class]
        
        n_samples = len(minority_samples)
        if len(majority_samples) < n_samples:
            n_samples = len(majority_samples)
        
        downsampled_majority = majority_samples.sample(
            n=n_samples, 
            replace=len(majority_samples) < len(minority_samples),
            random_state=bag*BAGS+fold
        )
        
        sex_train_idx = minority_samples.index.union(downsampled_majority.index)
        X_important_features_sex = important_features_sex_filtered.iloc[sex_train_idx]
        y_important_features_sex_balanced = y_sex.iloc[sex_train_idx]
        X_valid_sex = important_features_sex_filtered.iloc[valid_idx]
        y_valid_sex = y_sex.iloc[valid_idx]

        # Model parameters
        model_params = {
            'n_estimators': 300,
            'learning_rate': 0.175, 
            'metric': 'cross_entropy_lambda',
            'objective': None,
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'reg_alpha': 0.0,
            'reg_lambda': 0.0009,
            'early_stopping_round': 50,
            'feature_fraction': 0.1,
            'subsample': 0.8,
            'verbose': -1
        }

        model_sep_sex = LGBMClassifier(**model_params)
        model_sep_sex.fit(X_important_features_sex, y_important_features_sex_balanced,
                          eval_set=[(X_valid_sex, y_valid_sex)])

        sex_pred = model_sep_sex.predict(X_valid_sex)
        fold_f1_sex.append(f1_score(y_valid_sex, sex_pred, average='macro'))
        
        bag_pred_sex[valid_idx] = model_sep_sex.predict_proba(X_valid_sex)[:, 1]
        models_sep_sex[bag].append(model_sep_sex)

    bag_avg_f1_sex = np.mean(fold_f1_sex)
    bag_f1_sex.append(bag_avg_f1_sex)
    print(f"\nBag {bag+1} Avg F1 - Sex: {bag_avg_f1_sex:.4f}")
    oof_sex += bag_pred_sex / BAGS

# Final evaluation
final_f1_sex = f1_score(y_sex, (oof_sex > 0.5).astype(int), average='macro')
print('\n' + '#'*50)
print(f"Final Ensemble F1 - Sex: {final_f1_sex:.4f}")
print(f"Mean Bag F1 - Sex: {np.mean(bag_f1_sex):.4f} ± {np.std(bag_f1_sex):.4f}")


#########################
### Bag 1
#########################
=> Fold 1, => Fold 2, => Fold 3, => Fold 4, => Fold 5, 
Bag 1 Avg F1 - Sex: 0.7486
#########################
### Bag 2
#########################
=> Fold 1, => Fold 2, => Fold 3, => Fold 4, => Fold 5, 
Bag 2 Avg F1 - Sex: 0.7508
#########################
### Bag 3
#########################
=> Fold 1, => Fold 2, => Fold 3, => Fold 4, => Fold 5, 
Bag 3 Avg F1 - Sex: 0.7543
#########################
### Bag 4
#########################
=> Fold 1, => Fold 2, => Fold 3, => Fold 4, => Fold 5, 
Bag 4 Avg F1 - Sex: 0.7350
#########################
### Bag 5
#########################
=> Fold 1, => Fold 2, => Fold 3, => Fold 4, => Fold 5, 
Bag 5 Avg F1 - Sex: 0.7443

##################################################
Final Ensemble F1 - Sex: 0.7909
Mean Bag F1 - Sex: 0.7466 ± 0.0066


### ADHD

In [ ]:
# Drop only the columns from `trf` that are present in `train`
columns_to_drop = [col for col in trf.columns if col in train.columns]
train_adhd = train.drop(columns=columns_to_drop)

# Drop the target columns
train_adhd = train_adhd.drop(columns=target_cols)

### LGBM WITH DOWNSAMPLING 

In [ ]:
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from scipy.stats import hmean

models = {}
bag_f1_scores = []
stds = []
F1s = []

BAGS = 5
FOLDS = 5
target_col = 'ADHD_Outcome'

# Prepare data
X = train.drop(columns=[target_col, 'participant_id', 'Sex_F'])
X = pd.get_dummies(X, drop_first=True)
y = train[target_col]

for bag in range(BAGS):
    print('#'*25)
    print(f'### Bag {bag+1}')
    print('#'*25)
    
    models[bag] = []
    bag_pred = np.zeros(len(X))
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=bag*FOLDS)
    fold_f1s = []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y), 1):
        print(f'=> Fold {fold}, ', end='')

        # Your exact downsampling technique
        y_train = y.iloc[train_idx]
        class_counts = y_train.value_counts()
        
        if len(class_counts) > 1:
            majority_class = class_counts.idxmax()
            minority_class = 1 - majority_class
            majority_samples = y_train[y_train == majority_class]
            minority_samples = y_train[y_train == minority_class]
            
            n_samples = len(minority_samples)
            downsampled_majority = majority_samples.sample(
                n=n_samples, 
                replace=len(majority_samples) < len(minority_samples),
                random_state=bag*BAGS+fold
            )
            balanced_idx = minority_samples.index.union(downsampled_majority.index)
            X_train = X.iloc[balanced_idx]
            y_train_balanced = y.iloc[balanced_idx]
        else:
            X_train = X.iloc[train_idx]
            y_train_balanced = y.iloc[train_idx]

        # Model training (unchanged)
        model = LGBMClassifier(
            n_estimators=300,
            learning_rate=0.175,
            num_leaves=31,
            reg_alpha=0.0,
            reg_lambda=0.0009,
            feature_fraction=0.1,
            subsample=0.8,
            verbose=-1,
            class_weight='balanced'
        )
        model.fit(X_train, y_train_balanced)
        
        # Get predictions
        pred = model.predict(X.iloc[valid_idx])
        fold_f1 = f1_score(y.iloc[valid_idx], pred)
        fold_f1s.append(fold_f1)
        print(f"F1: {fold_f1:.4f}")
        
        # Store predictions
        bag_pred[valid_idx] = model.predict_proba(X.iloc[valid_idx])[:, 1]
        models[bag].append(model)
        
        # Store validation set std for this fold
        stds.append(y.iloc[valid_idx].std())
    
    # Bag-level results
    bag_avg_f1 = np.mean(fold_f1s)
    bag_f1_scores.append(bag_avg_f1)
    print(f"\nBag {bag+1} Avg F1: {bag_avg_f1:.4f}")
    oof_predictions += bag_pred / BAGS
    F1s.append(fold_f1s)

# Convert to numpy array for calculations
F1s = np.array(F1s)

# Final evaluation (aligned with your original validation structure)
print('\n' + '#'*50)
print("==== Overall Results ===")
print(f"Mean F1: {np.mean(F1s):.4f}")
print(f"F1 std: {np.std(F1s):.4f}")

print("\nStandard deviations of test sets:")
for i, std in enumerate(stds):
    print(f"Fold {i+1}: {std:.4f}")

# Final OOF score calculation
final_preds = (oof_predictions > 0.5).astype(int)
final_f1 = f1_score(y, final_preds)
print(f"\nFinal OOF F1: {final_f1:.4f}")

#########################
### Bag 1
#########################
=> Fold 1, F1: 0.8013
=> Fold 2, F1: 0.8013
=> Fold 2, F1: 0.7682
=> Fold 3, F1: 0.7682
=> Fold 3, F1: 0.7810
=> Fold 4, F1: 0.7810
=> Fold 4, F1: 0.8037
=> Fold 5, F1: 0.8037
=> Fold 5, F1: 0.8287

Bag 1 Avg F1: 0.7966
#########################
### Bag 2
#########################
=> Fold 1, F1: 0.8287

Bag 1 Avg F1: 0.7966
#########################
### Bag 2
#########################
=> Fold 1, F1: 0.7834
=> Fold 2, F1: 0.7834
=> Fold 2, F1: 0.8113
=> Fold 3, F1: 0.8113
=> Fold 3, F1: 0.7987
=> Fold 4, F1: 0.7987
=> Fold 4, F1: 0.7582
=> Fold 5, F1: 0.7582
=> Fold 5, F1: 0.8202

Bag 2 Avg F1: 0.7944
#########################
### Bag 3
#########################
=> Fold 1, F1: 0.8202

Bag 2 Avg F1: 0.7944
#########################
### Bag 3
#########################
=> Fold 1, F1: 0.7974
=> Fold 2, F1: 0.7974
=> Fold 2, F1: 0.8091
=> Fold 3, F1: 0.8091
=> Fold 3, F1: 0.7925
=> Fold 4, F1: 0.7925
=> Fold 4, F1: 0.7821
=> Fold 

### LGBM WITHOUT DOWNSAMPLING

In [ ]:
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

# Initialize storage
oof_predictions = np.zeros(len(train))
models = {}
bag_f1_scores = []
stds = []
F1s = []

BAGS = 5
FOLDS = 5
target_col = 'ADHD_Outcome'

# Prepare data
X = train.drop(columns=[target_col, 'participant_id', 'Sex_F'])
X = pd.get_dummies(X, drop_first=True)
y = train[target_col]

for bag in range(BAGS):
    print('#'*25)
    print(f'### Bag {bag+1}')
    print('#'*25)
    
    models[bag] = []
    bag_pred = np.zeros(len(X))
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=bag*FOLDS)
    fold_f1s = []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y), 1):
        print(f'=> Fold {fold}, ', end='')

        # Model training WITHOUT downsampling
        model = LGBMClassifier(
            n_estimators=300,
            learning_rate=0.175,
            num_leaves=31,
            reg_alpha=0.0,
            reg_lambda=0.0009,
            feature_fraction=0.1,
            subsample=0.8,
            verbose=-1,
            class_weight='balanced'  # Handling imbalance through class weights
        )
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        
        # Get predictions
        pred = model.predict(X.iloc[valid_idx])
        fold_f1 = f1_score(y.iloc[valid_idx], pred)
        fold_f1s.append(fold_f1)
        print(f"F1: {fold_f1:.4f}")
        
        # Store predictions
        bag_pred[valid_idx] = model.predict_proba(X.iloc[valid_idx])[:, 1]
        models[bag].append(model)
        
        # Store validation set std for this fold
        stds.append(y.iloc[valid_idx].std())
    
    # Bag-level results
    bag_avg_f1 = np.mean(fold_f1s)
    bag_f1_scores.append(bag_avg_f1)
    print(f"\nBag {bag+1} Avg F1: {bag_avg_f1:.4f}")
    oof_predictions += bag_pred / BAGS
    F1s.append(fold_f1s)

# Final evaluation
print('\n' + '#'*50)
print("==== Overall Results ===")
print(f"Mean F1 across bags: {np.mean(bag_f1_scores):.4f} ± {np.std(bag_f1_scores):.4f}")

print("\nFold-wise standard deviations:")
for i, std in enumerate(stds[:FOLDS]):  # Show first fold's std as representative
    print(f"Fold {i+1}: {std:.4f}")

# Final OOF score
final_preds = (oof_predictions > 0.5).astype(int)
final_f1 = f1_score(y, final_preds)
print(f"\nFinal OOF F1: {final_f1:.4f}")

#########################
### Bag 1
#########################
=> Fold 1, F1: 0.8453
=> Fold 2, F1: 0.8385
=> Fold 3, F1: 0.8689
=> Fold 4, F1: 0.8556
=> Fold 5, F1: 0.8501

Bag 1 Avg F1: 0.8517
#########################
### Bag 2
#########################
=> Fold 1, F1: 0.8478
=> Fold 2, F1: 0.8315
=> Fold 3, F1: 0.8500
=> Fold 4, F1: 0.8453
=> Fold 5, F1: 0.8595

Bag 2 Avg F1: 0.8468
#########################
### Bag 3
#########################
=> Fold 1, F1: 0.8485
=> Fold 2, F1: 0.8476
=> Fold 3, F1: 0.8289
=> Fold 4, F1: 0.8338
=> Fold 5, F1: 0.8595

Bag 3 Avg F1: 0.8437
#########################
### Bag 4
#########################
=> Fold 1, F1: 0.8329
=> Fold 2, F1: 0.8541
=> Fold 3, F1: 0.8466
=> Fold 4, F1: 0.8232
=> Fold 5, F1: 0.8556

Bag 4 Avg F1: 0.8425
#########################
### Bag 5
#########################
=> Fold 1, F1: 0.8418
=> Fold 2, F1: 0.8871
=> Fold 3, F1: 0.8509
=> Fold 4, F1: 0.8556
=> Fold 5, F1: 0.8315

Bag 5 Avg F1: 0.8534

#############################

## ONLY TRAIN_ADHD

here i traind the data with only TRAIN_ADHD and those the features without Fmri data

In [ ]:
train_adhd

,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,2019,4,1.0,0.0,3.0,21.0,45.000000,17.903324,32.871543,0.818599,-0.200460,-0.619845,0.466715,0.870819,-1.195718,0.676164,0.492507,0.459262,0.737479,0.776497,0.823227,0.320435,0.880342,0.400556,-0.132149,0.603281,1.052928
1,2017,1,0.0,9.0,2.0,21.0,0.000000,21.000000,45.000000,0.662301,0.277591,-0.619845,-0.372253,-0.985375,2.118640,2.233637,1.479851,1.451997,1.199700,0.776497,1.309451,0.320435,0.880342,0.686599,0.348525,0.123977,1.088973
2,2017,1,1.0,2.0,2.0,9.0,0.000000,12.387745,17.897575,0.549071,0.277591,-0.619845,2.144650,-0.572888,0.009503,1.844269,0.821621,0.459262,1.815994,2.167026,0.580116,2.108335,0.522620,2.688906,2.271221,-0.355327,0.773597
3,2018,1,3.0,8.0,2.0,18.0,10.000000,18.000000,0.000000,0.683936,0.277591,-0.619845,-0.651909,0.458331,-0.894413,0.676164,0.821621,-1.029841,-1.111402,-1.077543,-1.121665,-1.467465,-0.908267,-0.743619,-0.132149,-0.834631,-0.566931
4,2018,1,0.0,1.0,2.0,12.0,0.000000,13.845904,22.432518,-1.203370,0.277591,3.157189,-0.372253,-0.985375,1.214724,-0.881308,-0.494837,1.948365,1.661920,2.167026,1.795674,1.393175,1.238063,0.972643,-0.612823,-1.793238,-1.572278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,2019,4,1.0,1.0,3.0,12.0,15.136452,15.000000,5.000000,0.818599,0.277591,-0.619845,0.187059,-0.779131,0.009503,-0.686624,-1.153066,1.451997,1.199700,1.240007,1.309451,0.320435,0.880342,0.686599,-0.132149,0.123977,0.018928
1209,2018,1,0.0,0.0,3.0,21.0,40.000000,21.000000,40.000000,-0.259717,0.277591,-0.619845,1.305682,0.458331,0.913419,-0.297256,-0.165722,-0.037106,-1.111402,-1.077543,-0.635442,-1.467465,-0.908267,-1.315707,-1.093497,-0.355327,-1.327849
1210,2018,3,2.0,3.0,3.0,21.0,40.000000,21.000000,35.000000,-2.012158,0.277591,-0.619845,1.585338,-1.191619,0.612114,0.676164,-0.165722,-0.037106,0.275259,0.312987,0.337004,-0.394725,0.522620,0.114512,-0.132149,0.603281,-0.432709
1211,2019,4,0.0,1.0,3.0,18.0,35.000000,18.000000,45.000000,0.818599,0.277591,-0.619845,-1.211221,1.077062,1.516030,-1.075992,1.479851,-0.533473,-0.341035,-1.077543,0.093893,0.320435,0.522620,-0.743619,-0.132149,-0.355327,-0.994939


In [ ]:
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

# Initialize storage
oof_predictions = np.zeros(len(train_adhd))
models = {}
bag_f1_scores = []
stds = []
F1s = []

BAGS = 5
FOLDS = 5
target_col = 'ADHD_Outcome'

# Prepare data
X = train_adhd
X = pd.get_dummies(X, drop_first=True)
y = train[target_col]

for bag in range(BAGS):
    print('#'*25)
    print(f'### Bag {bag+1}')
    print('#'*25)
    
    models[bag] = []
    bag_pred = np.zeros(len(X))
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=bag*FOLDS)
    fold_f1s = []
    
    for fold, (train_adhd_idx, valid_idx) in enumerate(skf.split(X, y), 1):
        print(f'=> Fold {fold}, ', end='')

        # Model train_adhding WITHOUT downsampling
        model_adhd = LGBMClassifier(
            n_estimators=100,
            learning_rate=0.175,
            num_leaves=31,
            reg_alpha=0.0,
            reg_lambda=0.0009,
            feature_fraction=0.1,
            subsample=0.8,
            verbose=-1,
            class_weight='balanced'  # Handling imbalance through class weights
        )
        model_adhd.fit(X.iloc[train_adhd_idx], y.iloc[train_adhd_idx])
        
        # Get predictions
        pred = model_adhd.predict(X.iloc[valid_idx])
        fold_f1 = f1_score(y.iloc[valid_idx], pred)
        fold_f1s.append(fold_f1)
        print(f"F1: {fold_f1:.4f}")
        
        # Store predictions
        bag_pred[valid_idx] = model_adhd.predict_proba(X.iloc[valid_idx])[:, 1]
        models[bag].append(model_adhd)
        
        # Store validation set std for this fold
        stds.append(y.iloc[valid_idx].std())
    
    # Bag-level results
    bag_avg_f1 = np.mean(fold_f1s)
    bag_f1_scores.append(bag_avg_f1)
    print(f"\nBag {bag+1} Avg F1: {bag_avg_f1:.4f}")
    oof_predictions += bag_pred / BAGS
    F1s.append(fold_f1s)

# Final evaluation
print('\n' + '#'*50)
print("==== Overall Results ===")
print(f"Mean F1 across bags: {np.mean(bag_f1_scores):.4f} ± {np.std(bag_f1_scores):.4f}")

print("\nFold-wise standard deviations:")
for i, std in enumerate(stds[:FOLDS]):  # Show first fold's std as representative
    print(f"Fold {i+1}: {std:.4f}")

# Final OOF score
final_preds = (oof_predictions > 0.5).astype(int)
final_f1 = f1_score(y, final_preds)
print(f"\nFinal OOF F1: {final_f1:.4f}")

#########################
### Bag 1
#########################
=> Fold 1, F1: 0.8363
=> Fold 2, F1: 0.7768
=> Fold 3, F1: 0.8480
=> Fold 4, F1: 0.8448
=> Fold 5, F1: 0.8343

Bag 1 Avg F1: 0.8280
#########################
### Bag 2
#########################
=> Fold 1, F1: 0.7977
=> Fold 2, F1: 0.8131
=> Fold 3, F1: 0.8443
=> Fold 4, F1: 0.8503
=> Fold 5, F1: 0.8496

Bag 2 Avg F1: 0.8310
#########################
### Bag 3
#########################
=> Fold 1, F1: 0.8204
=> Fold 2, F1: 0.8446
=> Fold 3, F1: 0.8276
=> Fold 4, F1: 0.8353
=> Fold 5, F1: 0.8354

Bag 3 Avg F1: 0.8326
#########################
### Bag 4
#########################
=> Fold 1, F1: 0.8094
=> Fold 2, F1: 0.8174
=> Fold 3, F1: 0.8319
=> Fold 4, F1: 0.8546
=> Fold 5, F1: 0.8494

Bag 4 Avg F1: 0.8325
#########################
### Bag 5
#########################
=> Fold 1, F1: 0.8314
=> Fold 2, F1: 0.8408
=> Fold 3, F1: 0.8300
=> Fold 4, F1: 0.8319
=> Fold 5, F1: 0.8085

Bag 5 Avg F1: 0.8285

#############################

### predictions

In [ ]:
# Drop only the columns from `trf` that are present in `train`
columns_to_drop = [col for col in trf.columns if col in train.columns]
test_adhd = test.drop(columns=columns_to_drop)

# Drop the target columns
test_adhd = train_adhd

In [ ]:
test_adhd['paticipant_id'] = test['participant_id'] 

In [ ]:
def predict_adhd(test_adhd, models, feature_columns=None, threshold=0.5):
    """
    Make predictions using the trained bagged LGBM model for ADHD
    
    Args:
        test_adhd: DataFrame containing the same features as training data
        models: Dictionary of trained models from your training code
        feature_columns: List of feature names used in training (optional)
        threshold: Classification cutoff (default=0.5)
    
    Returns:
        DataFrame with predictions and confidence metrics
    """
    # Preprocess new data
    X_new = pd.get_dummies(test_adhd, drop_first=True)
    
    # If feature columns not provided, infer from first model
    if feature_columns is None:
        feature_columns = models[0][0].feature_name_
    
    # Ensure feature alignment
    missing_features = set(feature_columns) - set(X_new.columns)
    extra_features = set(X_new.columns) - set(feature_columns)
    
    for f in missing_features:
        X_new[f] = 0  # Add missing features with 0
    X_new = X_new[feature_columns]  # Reorder and remove extras
    
    # Initialize storage
    bag_predictions = np.zeros((len(X_new), BAGS))
    oof_predictions = np.zeros(len(X_new))
    
    # Make predictions from each bag
    for bag in range(BAGS):
        # Average predictions across folds within bag
        bag_pred = np.zeros(len(X_new))
        for fold_model in models[bag]:
            bag_pred += fold_model.predict_proba(X_new)[:, 1] / FOLDS
        bag_predictions[:, bag] = bag_pred
        oof_predictions += bag_pred / BAGS
    
    # Calculate uncertainty metrics
    pred_std = np.std(bag_predictions, axis=1)
    confidence = 1 - pred_std  # Higher value = more confident
    
    # Ensure 'participant_id' exists in test_adhd
    if 'participant_id' not in test_adhd.columns:
        raise KeyError("'participant_id' column is missing in the input data.")

    # Create results DataFrame
    results = pd.DataFrame({
        'participant_id': test_adhd['participant_id'],
        'adhd_probability': oof_predictions,
        'adhd_prediction': (oof_predictions > threshold).astype(int),
        'prediction_std': pred_std,
        'confidence_score': confidence,
        'bag_agreement': (bag_predictions > threshold).mean(axis=1)
    })
    
    return results

# Example usage -------------------------------------------------

# 1. Get feature columns (run after training)
feature_columns = models[0][0].feature_name_  # From first model

# 2. Ensure 'participant_id' column exists in test_adhd
if 'participant_id' not in test_adhd.columns:
    test_adhd['participant_id'] = test['participant_id']  # Add 'participant_id' from the original test DataFrame

# 3. Make predictions
predictions = predict_adhd(test_adhd, models, feature_columns)

# 4. Save/use results
print(predictions.head())
# predictions.to_csv('adhd_predictions.csv', index=False)

  participant_id  adhd_probability  adhd_prediction  prediction_std  \
0   Cfwaf5FX7jWK          0.986018                1        0.004542   
1   vhGrzmvA3Hjq          0.966732                1        0.009391   
2   ULliyEXjy4OV          0.294029                0        0.011514   
3   LZfeAb1xMtql          0.156272                0        0.030929   
4   EnFOUv0YK1RG          0.991727                1        0.001153   

   confidence_score  bag_agreement  
0          0.995458            1.0  
1          0.990609            1.0  
2          0.988486            0.0  
3          0.969071            0.0  
4          0.998847            1.0  


In [ ]:
# Drop only the columns from `trf` that are present in `train`
columns_to_drop = [col for col in trf.columns if col in test.columns]
test_adhd = test.drop(columns=columns_to_drop)

# Drop the target columns
test_adhd = test_adhd

### submission

In [ ]:
# Ensure all arrays have the same length
if len(test_ids) == len(test_preds_sex_labels) == len(adhd_predictions_binary):
    # Create a DataFrame for submission
    submission = pd.DataFrame({
        'participant_id': test_ids,
        'ADHD_Outcome': adhd_predictions_binary,
        'Sex_F': test_preds_sex_labels,

    })

    # Save the submission file
    submission.to_csv('submission_adhd_84_sex_68.csv', index=False)
    print("Submission file created successfully!")
else:
    print("Error: Length mismatch between test_ids, test_preds_sex_labels, and adhd_prediction.")

Submission file created successfully!


In [ ]:
from scipy.stats import hmean

# Using harmonic mean of bag scores (though mathematically similar for single metric)
final_f1_hmean = hmean(bag_f1_scores)  
print(f"Final ADHD F1 (harmonic mean): {final_f1_hmean:.4f}")

Final ADHD F1 (harmonic mean): 0.8476
